In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime, timedelta
import csv

## Objective:
-Import the datasets. Still need the extra dataset. Includes triggers up to dec. 21

-datetime helpful docs: https://www.w3schools.com/python/python_datetime.asp

-Create algorithm that searches for matches that are +/- 10 second apart and remove the rest (maybe two seconds)

-First match found at 2018, 7, 11, 17, 2, 4

-Complete match list found for trigB --> trigB_match

//TODO: 

- Update list up until 2022
- Wrap functions around the algorithms so that the same can be done for trigC etc.
- Create dataframe with the result
-Access the relevant files found in trigB_match list
-Do spectral analysis on the data (counts/bin)


### Notes from stackoverflow:
np.where() implementation: https://stackoverflow.com/questions/35714902/find-location-of-pair-of-elements-in-two-arrays-in-numpy

## Importing data

In [13]:
#Importing and redefining the dataframes
#Not loading the trigger list for 2021 just yet

#Old IPN dataset
ipn_old_data = pd.read_csv("./IPN/trigIPN.csv", sep="|")
ipn_old_df = pd.DataFrame(ipn_old_data)
ipn_old_df.drop(columns= ['Unnamed: 0', 'Unnamed: 2'], axis=1, inplace=True) #dropping unwanted columns
ipn_old_df.rename(columns={ipn_old_df.columns[0]:"datetime"}, inplace = True)

#New IPN dataset
ipn_new_data = pd.read_csv("./IPN/ipn_supp.txt",sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
ipn_new_df = pd.DataFrame(ipn_new_data)
#Dropping unwanted columns
ipn_new_df = ipn_new_df.drop(["GRB_name_Fermi",'ra(°J2000)', 'decl(°J2000)',
       'pos_error(°1-sigma)', 'T90(s)', 'T90_error(s)', 'T90_start(UTC)',
       'fluence(erg/cm^2)', 'fluence_error(erg/cm^2)', 'redshift', 'T100(s)',
       'GBM_located', 'mjd(T0day)'], 1)

#HED 
trigB_data = pd.read_csv("./ASIM/trigB.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigB = pd.DataFrame(trigB_data)
trigB.drop(columns=["######"], inplace=True) #dropping unwanted columns
trigB.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", "Corr":"corr"}, inplace=True) #renaming columns

trigB_21_data = pd.read_csv("./ASIM/trigB_2021.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigB_21 = pd.DataFrame(trigB_21_data)
trigB_21.drop(columns=["######"], inplace=True)
trigB_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time" ,"Corr":"corr"}, inplace=True)

#LED
trigC_data = pd.read_csv("./ASIM/trigC.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigC = pd.DataFrame(trigC_data)
trigC.drop(columns=["######"], inplace=True)
trigC.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time" ,"Corr":"corr"}, inplace=True)

trigC_21_data = pd.read_csv("./ASIM/trigC_2021.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigC_21 = pd.DataFrame(trigC_21_data)
trigC_21.drop(columns=["######"], inplace=True)
trigC_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", 'Corr':"corr" }, inplace=True)

#MMIA (visible)
trigM_data = pd.read_csv("./ASIM/trigM.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigM = pd.DataFrame(trigM_data)
trigM.drop(columns=["######"], inplace=True)
trigM.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", "Corr":"corr" }, inplace=True)

trigM_21_data = pd.read_csv("./ASIM/trigM_2021.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigM_21 = pd.DataFrame(trigM_21_data)
trigM_21.drop(columns=["######"], inplace=True)
trigM_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", "Corr":"corr" }, inplace=True)

## Concatenating ASIM dataframes

In [5]:
def find_index_21(l):
    """
    Function for finding the indecies to slice. Assuming l is sorted by date from first to last.
    Finding index of first entry of '2021-Jan-01'.
    
    Parameters:
    -----
    l <array>
    
    Returns:
    -----
    Returning index for the first entry of '2021-Jan-01' """
    
    
    i = l.tolist().index('2021-Jan-01')
    return i

In [6]:
slice_index = []
for df in [trigB,trigC,trigM]:
    temp_date_list = df["date"].values
    slice_index.append(find_index_21(temp_date_list))
slice_index

[110410, 79065, 237671]

In [8]:
#trig*_20 contains rows to the end of 2020
#trig*_master contains rows to the end of 2021
trigB_20 = trigB.drop(np.arange(slice_index[0],len(trigB))) 
trigB_master = pd.concat([trigB_20,trigB_21])

trigC_20 = trigC.drop(np.arange(slice_index[1],len(trigC)))
trigC_master = pd.concat([trigC_20,trigC_21])

trigM_20 = trigM.drop(np.arange(slice_index[2],len(trigM)))
trigM_master = pd.concat([trigM_20,trigM_21])

In [9]:
trigB_master,trigC_master,trigM_master

(              date             time      corr
 0      2018-Jun-01  12:46:08.843153   -191662
 1      2018-Jun-01  12:51:01.726322   -192770
 2      2018-Jun-01  13:06:13.113589   -196203
 3      2018-Jun-01  13:12:04.912824   -197526
 4      2018-Jun-01  13:14:19.594102   -198029
 ...            ...              ...       ...
 37716  2021-Dec-31  23:05:01.249646    100055
 37717  2021-Dec-31  23:33:47.967067  --------
 37718  2021-Dec-31  23:36:28.826496  --------
 37719  2021-Dec-31  23:38:39.674400  --------
 37720  2021-Dec-31  23:42:49.746266  --------
 
 [148131 rows x 3 columns],
               date             time      corr
 0      2018-Jun-01  05:10:05.271627    -88533
 1      2018-Jun-01  05:12:03.803744    -88987
 2      2018-Jun-01  05:15:54.113081    -89864
 3      2018-Jun-01  05:16:36.442279    -90024
 4      2018-Jun-01  09:54:36.077906   -153002
 ...            ...              ...       ...
 32628  2021-Dec-31  23:24:30.527051  --------
 32629  2021-Dec-31  23:27:58.

## IPN Handling

### IPN_old

In [21]:
#Old IPN datetime list
ipn_dt_temp = []
ipn_dict = ipn_old_df.to_dict("records")
for row in ipn_dict:
    datetime_str = row["datetime"]
    datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    ipn_dt_temp.append(datetime_obj)
    
ipn_old_dt = np.asarray(ipn_dt_temp)
ipn_old_dt.shape

(1176,)

### IPN new

In [31]:
# Zero-padding hour in times

def zero_padding(l):
    """Function for zero padding times in IPN new
    Parameters:
    -----
    l <array> containing string of times

    Returns:
    -----
    new_times <array> containing string of zero-padded times
    """
    new_times = []

    for time in l:
        if time.index(":") < 2:
            time = "0" + time
            new_times.append(time)
        else:
            new_times.append(time)
    return np.array(new_times)

In [32]:
#Vectorization of names and times from dataframe.
names = ipn_new_df["GRB_name"].values
times = ipn_new_df["T0(UTC)"].values  # Hour is not zero padded.
new_times = zero_padding(times)

In [34]:
# Defining datetime objects in ipn_new_list.
# Problem: some of the times have microsecond resolution while other dont
ipn_new_list = []
if len(names) == len(new_times):
    for i in np.arange(0, len(names)):
        date = names[i][3:9]
        time = new_times[i]

        # Managing the microsecond resolution
        if len(time) > 8:
            dt_obj = datetime.strptime(date + " " + time, "%y%m%d %H:%M:%S.%f")
        else:
            dt_obj = datetime.strptime(date + " " + time, "%y%m%d %H:%M:%S")
            
        ipn_new_list.append(dt_obj)
else:
    print("Unequal length of arrays")
    
#Earliest entry in the ASIM data 2018,6,1,12,46,8
# Including only entries up to 1142. That means entries in IPN starts from 2018, 5, 29, 8, 29, 14

ipn_new_dt = np.asarray(ipn_new_list[:1142][::-1]) #Slicing the list. Returning in reverse order (decending)
ipn_new_dt.shape

(1142,)

In [ ]:
#Function for stripping the microseconds from an datetime object
def strip_microseconds(l):
    """
    Strip microseconds from datetime objects in list containing a tuple of dt objects and int. Returning string
    objects. Seperating them in two arrays
    
    Parameters
    ----------
    l : list of datetime objects
        The list containing the objects.
    
    Returns
    -------
    updated_list : array of stripped datetime objects as string and array of trig_id
    
    Raises
    ------
    TypeError
        If the input is not a type list or array
    """
    time = []
    trig_id = []
    try:
        for tup in l:
            dt = tup[0].strftime("%Y-%m-%d %H:%M:%S")
            time.append(dt)
            trig_id.append(tup[1])
            
        return np.array((time,trig_id))
    
    except TypeError:
        print("Input is not the correct type <List> or <nd array> or entries is not tuple")

In [ ]:
#Important to keep the microseconds? Check the search algo
ipn_old_updated = strip_microseconds(matches_old)
ipn_new_updated = strip_microseconds(matches_new)

## ASIM datetime correction, vectorization

In [41]:
# Function for correcting date and time in ASIM data. Returning ndarray that contains datetime objects

def corr_dt(dfs):  # Correcting times from ASIM data and returning nested array of datetime objects
    
    """   This function corrects the time from ASIM data and returns datetimeobjects
    The correction is done by subtracting the correction from the original time.
    The correction is given as a string.
    The function takes a list of dataframes as input.
    The function returns a list of arrays containing the corrected datetime objects.
    
    Parameters
    ----------
    MAYBE LISTS CONTAINING STRING OF tuple(DATE,TIME)?
    
    dfs : list of dataframes
        The dataframes containing the data.
    
    Returns
    -------
    trig_dt : list of arrays
        The corrected datetime objects.
        trig_dt[0] --> trigB
        trig_dt[1] --> trigC
        trig_dt[2] --> trigM
    
    Raises
    ------
    ValueError
        If the lists are not the same length.
    """
    trig_dt = []
    
    for df in dfs:
        temp_dt = []
        # Vectorization of columns
        date = df["date"].values  # date given as string.
        time = df["time"].values  # time given as string
        corr = df["corr"].values  # correction given as string
        try:
            if len(date) and len(time) != len(corr):
                raise ValueError
        except:
            raise ValueError("Lists are not the same length")
        else:
            for i in np.arange(0, len(corr)):  # Iterating over the vectors
                if corr[i] == "--------":  # No correction needed. Appending the datetime object
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")
                    temp_dt.append(org_dt)
                    
                elif corr[i][0] == "-":  # If it's a "-" in front; correction is added
                    # formatting the datetime object
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")  # Original datetime

                    micro_corr = int(corr[0][1:])

                    # new corrected datetime. Timedelta ccounts for changes in seconds also
                    new_dt = org_dt + timedelta(microseconds=micro_corr)
                    temp_dt.append(new_dt)
                else:
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")  # Original datetime

                    micro_corr = int(corr[0][1:])

                    # new corrected datetime. Timedelta ccounts for changes in seconds also
                    new_dt = org_dt - timedelta(microseconds=micro_corr)
                    temp_dt.append(new_dt)
                    
            trig_dt.append(temp_dt)

    trig_dt = np.array(trig_dt, dtype=object)
    return trig_dt

In [43]:
#Callig corr_dt with a list containing the dataframes from ASIM
trig_dt = corr_dt([trigB_master,trigC_master,trigM_master])

#Creating array instances in trig_dt
for i in np.arange(0,len(trig_dt)):
    trig_dt[i] = np.asarray(trig_dt[i])

## ASIM and IPN search

In [46]:
#Most compact algorithm for searching: np.where()
#Storing matches in the match list. Stored as a tuple containing (datetime IPN, index trig_B)

# TODO: wrap a function around it so it can take in several triggers (trigB,trigC etc..)
def find_match(trig_arr):
    """
    Parameters:
    ------
    trig_arr <array> <datetime>
    np.where returns: An array with elements from x where condition is True, and elements from y elsewhere.
    
    elements in matches_* contains tuples for indecies in IPN list and trig_arr. """
    matches_old = [] 

    for i in ipn_old_dt:
        #Searching for matches that are +/- 10 seconds from the IPN trigger
        mask = np.where((i-timedelta(seconds=10) <= trig_arr) & (trig_arr <= i + timedelta(seconds=10)))
        if len(mask[0]) == 0:
            continue
        else:
            matches_old.append((i,mask[0]))

    matches_new = []
    for i in ipn_new_dt:
        #Searching for matches that are +/- 10 seconds from the IPN trigger
        mask = np.where((i-timedelta(seconds=10) <= trig_arr) & (trig_arr <= i + timedelta(seconds=10)))
        #print(mask)
        if len(mask[0]) == 0:
            continue
        else:
            matches_new.append((i,mask[0]))
        
    return (matches_old, matches_new)

In [47]:
master_match = []
for i in trig_dt:
    tup_match = find_match(i)
    master_match.append(tup_match)
    
#master_match[n] - refers to type of trigger (trigB, trigC, trigM)
#master_match[n][n] - refers to old or new ipn trigger list [0]: old , [1]: new
#master_match[n][n][n] - refers to tuple containing (ipn dt, array[index]) where index is the dt in trig_dt[n][index]
#master_match[n][n][n][n] - selecting desired element in tuple

In [77]:
master_match

[([(datetime.datetime(2018, 7, 11, 17, 2, 2), array([2852])),
   (datetime.datetime(2018, 7, 20, 14, 21, 44), array([3368, 3369])),
   (datetime.datetime(2018, 8, 9, 13, 12, 4), array([4831])),
   (datetime.datetime(2018, 8, 9, 14, 37, 3), array([4853])),
   (datetime.datetime(2018, 9, 10, 3, 44, 16), array([11365])),
   (datetime.datetime(2018, 10, 17, 20, 54, 15), array([18945])),
   (datetime.datetime(2018, 12, 22, 20, 11, 34), array([30144])),
   (datetime.datetime(2019, 2, 6, 3, 49, 23), array([43589])),
   (datetime.datetime(2019, 2, 16, 11, 52, 26), array([45481])),
   (datetime.datetime(2019, 2, 18, 19, 27, 44), array([45987])),
   (datetime.datetime(2019, 3, 5, 13, 5, 15), array([49414, 49426])),
   (datetime.datetime(2019, 3, 23, 21, 5, 21), array([53366])),
   (datetime.datetime(2019, 4, 11, 9, 45, 56), array([55408])),
   (datetime.datetime(2019, 5, 12, 14, 40, 9), array([58637])),
   (datetime.datetime(2019, 6, 6, 1, 55, 7), array([61032])),
   (datetime.datetime(2019, 7, 

## Branching
Branching into matches in old ipn dataset and matches in new ipn dataset.

Export into a dataframe containing IPN dt trigger | trig* dt trigger



How to access data from master_match:


- master_match[n] - refers to type of trigger (trigB, trigC, trigM)

- master_match[n][n] - refers to old or new ipn trigger list [0]: old , [1]: new

- master_match[n][n][n] - refers to tuple containing (ipn dt, array[index]) where index is the dt in trig_dt[n][index]

- master_match[n][n][n][n] - selecting desired element in tuple

In [81]:
match_trigB_old = master_match[0][0]
match_trigB_new = master_match[0][1]

match_trigC_old = master_match[1][0]
match_trigC_new = master_match[1][1]

match_trigM_old = master_match[2][0]
match_trigM_new = master_match[2][1]

In [182]:
# Creating dataframe for ipn old and ipn new. Columns should be ipn dt trigger, trigB, trigC,trigM
# if there isnt a trigger in the respected trigger, the entry should be None
# But first. Produce df of ipn --> trigB, trigC, trigM [indecies]. Just indecies because there are several in some triggers

trigB_old_dict = {"ipn old dt":[], "trigB index": []}
for i in match_trigB_old:
    trigB_old_dict["ipn old dt"].append(i[0])
    trigB_old_dict["trigB index"].append(i[1])
trigB_old_df = pd.DataFrame(trigB_old_dict)

trigB_new_dict = {"ipn new dt":[], "trigB index": []}
for i in match_trigB_new:
    trigB_new_dict["ipn new dt"].append(i[0])
    trigB_new_dict["trigB index"].append(i[1])
trigB_new_df = pd.DataFrame(trigB_new_dict)

trigC_old_dict = {"ipn old dt":[], "trigC index": []}
for i in match_trigC_old:
    trigC_old_dict["ipn old dt"].append(i[0])
    trigC_old_dict["trigC index"].append(i[1])
trigC_old_df = pd.DataFrame(trigC_old_dict)

trigC_new_dict = {"ipn new dt":[], "trigC index": []}
for i in match_trigC_new:
    trigC_new_dict["ipn new dt"].append(i[0])
    trigC_new_dict["trigC index"].append(i[1])
trigC_new_df = pd.DataFrame(trigC_old_dict)

trigM_old_dict = {"ipn old dt":[], "trigM index": []}
for i in match_trigM_old:
    trigM_old_dict["ipn old dt"].append(i[0])
    trigM_old_dict["trigM index"].append(i[1])
trigM_old_df = pd.DataFrame(trigM_old_dict)

trigM_new_dict = {"ipn new dt":[], "trigM index": []}
for i in match_trigM_new:
    trigM_new_dict["ipn new dt"].append(i[0])
    trigM_new_dict["trigM index"].append(i[1])
trigM_new_df = pd.DataFrame(trigM_new_dict)

In [223]:
trigB_old_indecies = trigB_old_df["trigB index"].values
trigB_old_dts = index_to_dt(0,trigB_old_indecies)

trigC_old_indecies = trigC_old_df["trigC index"].values
trigC_old_dts = index_to_dt(1,trigC_old_indecies)

trigM_old_indecies = trigM_old_df["trigM index"].values
trigM_old_dts = index_to_dt(2,trigM_old_indecies)


40


## Exporting matches from IPN and IPN_supp

In [ ]:
# Algorithm for saving the matches in a file:
trigB_match_file = open("trigB_match.txt", "w")
for i in np.arange(0,len(trigB_match)):
    for row in trigB_match[i]:
        np.savetxt(trigB_match_file, (str(row[0]), row[1]) , fmt='%s')

trigB_match_file.close()

### Graveyard

In [ ]:
#Function to filter duplicates
def filter_duplicates(l1,l2):
    """
    l1 --> ipn_old match
    l2 --> ipn_new match
    
    Converting l1,l2 to np.arrays to handle the filtering. (might not be neccecery)
    Checking if dt object in l2 is not present in l1. If so, then add the tuple to l1
    Parameters: l1,l2 <list> containing tuples
    ----
    Returns: l3 <array> of dt objects that are not in """
    
    found_time = []
    indecies = []
    for index, i in enumerate(ipn_new_updated[0]):
        if i not in ipn_old_updated:
            found_time.append(i)
            indecies.append(index)
        
    found_time = np.asarray(found_time)
    indecies = np.asarray(indecies)